# Airbnb Rio de Janeiro - Feature engineering

# 1. Introdução

Esse notebook propõe um processo de feature engineering para os [dados do Airbnb na cidade do Rio de Janeiro](https://insideairbnb.com/get-the-data/#:~:text=Rio%20de%20Janeiro%2C%20Rio%20de%20Janeiro%2C%20Brazil). 

A base de dados original foi previamente explorada e modificada durante a etapa de [análise exploratória (EDA)](https://github.com/BPRateiro/rio-airbnb-data-analysis/blob/main/notebooks/eda.ipynb). Os dados incluem informações detalhadas sobre a disponibilidade de listagens, características dos imóveis, interações de usuários, e preços. 

O objetivo principal é transformar essas informações para melhorar a capacidade preditiva de modelos de machine learning e facilitar análises mais profundas.

## 1.1 Configurações e bibliotecas

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
pd.set_option("future.no_silent_downcasting", True)
from summarytools import dfSummary
from sklearn.pipeline import Pipeline

from rio_airbnb.features import PreProcessamento, SepararOutliersIQR, RemoverFeaturesCorrelacionadas, OneHotImputer, KNNImputerTransformer

2024-10-01 09:38:30.578 | INFO     | rio_airbnb.config:<module>:11 - PROJ_ROOT path is: C:\Bruno\Projects\rio-airbnb-data-analysis


## 1.2 Leitura dos dados

In [2]:
df_train_raw = pd.read_parquet('../data/bronze/listings_train.parquet')

# Mesmas etapas do EDA
pipeline_eda = Pipeline(steps=[
    ('pre_processamento', PreProcessamento()),
    ('remover_outliers', SepararOutliersIQR(coluna='price')),
    ('remover_corr', RemoverFeaturesCorrelacionadas())
])
df_train_eda = pipeline_eda.fit_transform(df_train_raw)

# Cheque se o resultado é o mesmo
if df_train_eda.equals(pd.read_parquet('../data/bronze/feature_engineering_input.parquet')):
    print('O pipeline equivale às etapas do EDA')
else: 
    print('O pipeline NÃO equivale às etapas do EDA')

dfSummary(df_train_eda)

O pipeline equivale às etapas do EDA


No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,accommodates[int64],Mean (sd) : 3.7 (2.0)min < med < max:1.0 < 4.0 < 16.0IQR (CV) : 2.0 (1.9),16 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACkklEQVR4nO3dvY7aQBRA4TthIQaCLUwCHQ/ASiko6ZNyHzaPkIKWIrwBDUKykAkOCGOcJkm1YvFfPNc6Xz/YxQHpzkiMSdNUANu9q/sFgEc81f0C9xhjuiLSybn8kqbpqcz3QX2sDdUY0x2NRi+u6w7zrD8cDntjzDdibQZrQxWRjuu6w8VicfI875xlYRiGznK5HAZB0BERQm0Am0MVERHP887j8fhXjqXd0l8GtWGYggqEChUIFSoQKlR4c5hiLxM2uBsqe5mwxVu/qOxlwgoP7aOyl4m6MUxBBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQoVKb+5LkqQtIq4xJs9y93a7tUt+JShVWahRFLXjOP48nU5brVYr0///i4jEcewkSfJ8Pp+/i0iev2VHg1QW6uVyeXIcpzefz8+TyWSfdf1msxmu1+ve9XptVfF+0KXyS3v7/X6uiyr2+z0XVeAfhimoYP016HkVHOREuHXQKo0MteggJ8Ktg7ZpZKhFBzluHbRPI0P9K+8g9wfDnEUYpqACoUIFQoUKhAoVCBUqECpUaPT2VBEFT7Y41SoZob6i6MkWp1rlI9RXFDnZ4lSrGoR6R4GTLU61SsYwBRX4RbWMMaYrIp0CH9HIQY5QK1Bgx8Dxff+r53n9vM+uc5Ar+CW7+wV7KNQwDJ2sTz0ej46ISBRFzm636/3P9XU+e7vdDvLuGMRx7IjI82w2+zEYDH5mWSsicjqd3q9Wq09BEHw0xhyyri/I8X3/i+d5H/IsfusL9huPFhUdX7gJKgAAAABJRU5ErkJggg=="">",0(0.0%)
2,availability_365[int64],Mean (sd) : 200.4 (113.3)min < med < max:0.0 < 181.0 < 365.0IQR (CV) : 215.0 (1.8),366 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACeElEQVR4nO3aQW7aQBiG4X9KQAYkWygS7LhApS5YcogetkfoghNwA3ZICBnhxsIYd10VCNg4M5/1Pttoklm8Iv8M46qqMiB033xvAHjEm+8NoDucc0MzG9Rcfqqq6uPWDwkVL+GcG76/v/+M43hSZ/3hcNg7537dipVQ8SqDOI4ny+XyI0mS/JmFaZpGq9VqstvtBmZGqGhfkiT5dDr9U2Pp8N4POUxBAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCQtDP/Np8MQ4twYba9otxaAk2VGv5xTi0hByqmbX3YhxaOExBAqFCAqFCQvAzqi9cjYWFUK/gaiw8hHodV2OBIdQ7uBoLB4cpSCBUSOBff8d09baCUDuky7cVhNotnb2tINQO6uJtBYcpSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSGj1m6mGDyTiy+XSf+V+oKu1UJs+kCiKIirL8nue57/NrM7XgeiQNj9Raz+QMDPbbDaT9Xo9Op/PvTY2By2tP0qp+0Biv98H+0ACX+/TUBvMmV5nzLIs+2YWO+fqLPe2d+b66+6G2mTO9DljZlnWL4rix3w+7/V6vafHDl97Z66/7bNP1Npzps8Z83Q6vUVRNFosFvlsNts/u97j3pnrb3hoRq0zZ4YwY47HY8n5mLn+f7zwb0GD+bizM2ZThPpiTeZj3zNmyAdQQn2xJvOxzxkz9AMoobakznzsc8YM/QBKqPhHqAdQXk9BAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCwkNfoaZpGj37i4/HY2RmlmVZtN1uR1+5XvVvq+676fpH+voLqXKpBtyP0mAAAAAASUVORK5CYII="">",0(0.0%)
3,availability_60[int64],Mean (sd) : 43.3 (19.9)min < med < max:0.0 < 53.0 < 60.0IQR (CV) : 28.0 (2.2),61 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAC20lEQVR4nO3Yv26bUBSA8XPt2MGxxRVxSrYsHSN1sJSFB+iYh80jZPDQOUOXTtkiWRYoxMgE06lqh/wx90LMSb/fah9A4jPca1PXtQB9Nzj0BQD7ODr0BeDzMMZMRGTsOL6t63rz2oeEilYYYybz+fw6DMPIZT7LsrUx5ua1WAkVbRmHYRglSbKx1hZNBtM0DZbLZbRarcYiQqjonrW2iOP4yWF08taHbKagAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFChaP3vmCMmYjI2PH427quN46z+GCe9zrc7XajNq/nX2+GaoyZzOfz6zAMI5eDZ1m2NsbcEGv/+d7rsiyDqqoui6K4FZGnli/v3SfqOAzDKEmSjbW2aHLgNE2D5XIZrVarsYgQav8532sRkfv7++ju7u7k+fl52MXFvfvqFxGx1hZxHLv8SiYOMzgg13u9Xq87vdd7hQo9PNaZna4xfRFqBzw3JQMR2TnOBqenp9+ttdOmg12vMX0R6is8YnOOpaqqUZ7nX2ez2a/BYFA2nS/LMhCRy6urqx9nZ2dZk9mu15i+CPUFPjvgFmL5slgsfp6fn6+bnvtPbEEQlE3XmV2vMX192lB9/xOczWZxkiRZ0x1wG7FMp9NebmgOqdehHuL1K/J3vXZ8fHz72Z5MWnUaalVVIxEJjTEu414bA3F8/Yr0f732P+os1DzPR2VZfru4uBgOh8PGfyC3sTFwef2K8FTso85C3W63R0EQnCwWi4KNAXztFWqapkHTAz8+PjaeeUme58HDw8OJy7ldZn3nDzWr+dz79PUbWMI9qLerbfYAAAAASUVORK5CYII="">",0(0.0%)
4,bathrooms[float64],Mean (sd) : 1.4 (0.7)min < med < max:0.0 < 1.0 < 16.0IQR (CV) : 1.0 (1.9),20 distinct values,"<img src = ""data:i

# 2. One hot encoding

- Variáveis categóricas com múltiplas categorias com valores faltantes terão uma categoria adicional `nan` adicionada antes do processo de one hot encoding
- Variáveis binárias serão convertidas para valores numéricos (1 e 0)

De 41 colunas inicialmente, terminamos o processo com 44 colunas. 

In [3]:
df_train_onehot = OneHotImputer().fit_transform(df_train_eda)
df_train_onehot.shape

(24255, 44)

# 3. Imputação de missings

Utilizamos o KNNImputer para imputar missings em variáveis numéricas.
Normalizamos também o nome das colunas.

In [4]:
# KNNImputer
knn_imputer = KNNImputerTransformer(n_neighbors=5, copy=False, salvar=True, parquet_path='../data/silver/fe_train_output.parquet')
df_train_knn = knn_imputer.fit_transform(df_train_onehot)

if df_train_knn.isnull().values.any():
    print("O dataframe contém valores nulos.")
else:
    print("O dataframe não contém valores nulos.")

O dataframe não contém valores nulos.


# 4. Aplicação do pipeline no dataset de teste

Podemos aplicar todas etapas de uma vez no conjunto de teste com um único pipeline

In [5]:
# Criar o pipeline completo
pipeline_completo = Pipeline(steps=[
    ('pre_processamento', PreProcessamento()),             # Passo de pré-processamento
    ('remover_outliers', SepararOutliersIQR(coluna='price')), # Remover outliers
    ('remover_corr', RemoverFeaturesCorrelacionadas()),    # Remover colunas correlacionadas
    ('one_hot_encoding', OneHotImputer()),                 # Aplicar One-Hot Encoding
    ('knn_imputer', KNNImputerTransformer(salvar=True, parquet_path='../data/silver/fe_train_output.parquet')) # Aplicar KNN Imputer
])

df_train_knn = pipeline_completo.fit_transform(df_train_raw)

# Aplicar o pipeline no conjunto de teste
df_test_raw = pd.read_parquet('../data/bronze/listings_test.parquet')
pipeline_completo.named_steps['knn_imputer'].set_parquet_path('../data/silver/fe_test_output.parquet')
df_test_knn = pipeline_completo.transform(df_test_raw)

# 5. Conclusão

Aplicamos técnicas como one-hot encoding e imputação de valores ausentes com o KNNImputer, resultando em um conjunto de dados robusto e sem lacunas. Dessa forma estaremos melhor preparados para realizar análises preditivas e extrair insights significativos.